In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
block_size = 20
batch_size = 128
learning_rate = 3e-4
max_iter = 10000
eval_iters = 250

In [4]:
with open("/kaggle/input/wizard-of-oz-txt/wizard_of_oz.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [5]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
74


In [6]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

In [7]:
encode = lambda s: [string_to_int[ch] for ch in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

In [8]:
encode("hello")

[55, 52, 59, 59, 62]

In [9]:
decode([55, 52, 59, 59, 62])

'hello'

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)

In [11]:
print(data[500:600])

tensor([67, 55, 72,  1, 72, 62, 68, 61, 54, 66, 67, 52, 65,  1, 55, 48, 66,  1,
        48,  1, 70, 55, 62, 59, 52, 66, 62, 60, 52,  1, 48, 61, 51,  0, 56, 61,
        66, 67, 56, 61, 50, 67, 56, 69, 52,  1, 59, 62, 69, 52,  1, 53, 62, 65,
         1, 66, 67, 62, 65, 56, 52, 66,  1, 53, 48, 61, 67, 48, 66, 67, 56, 50,
         9,  1, 60, 48, 65, 69, 52, 59, 62, 68, 66,  1, 48, 61, 51,  1, 60, 48,
        61, 56, 53, 52, 66, 67, 59, 72,  0, 68])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
def get_batch(split):
  data = train_data if split == "train" else val_data
  idx  = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x, y

In [14]:
x, y = get_batch("train")

In [15]:
print("inputs : ")
print(x)
print("targets : ")
print(y)

inputs : 
tensor([[67,  0, 50,  ..., 53, 59, 48],
        [53, 62, 68,  ..., 56, 61,  1],
        [ 1, 50, 48,  ..., 66, 70, 52],
        ...,
        [33, 55,  9,  ...,  1, 56, 61],
        [66, 67,  9,  ..., 52, 66,  1],
        [10, 60, 62,  ...,  0,  0, 37]], device='cuda:0')
targets : 
tensor([[ 0, 50, 55,  ..., 59, 48, 63],
        [62, 68, 61,  ..., 61,  1, 48],
        [50, 48, 61,  ..., 70, 52, 65],
        ...,
        [55,  9,  1,  ..., 56, 61,  1],
        [67,  9,  0,  ..., 66,  1, 54],
        [60, 62, 65,  ...,  0, 37, 55]], device='cuda:0')


In [16]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [17]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
  
    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self.forward(index)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      index_next = torch.multinomial(probs, num_samples=1)
      index = torch.cat((index, index_next), dim=1)
    return index

In [18]:
model = BigramLanguageModel(vocab_size=len(chars))
model = model.to(device)

In [19]:
context = torch.zeros((batch_size, block_size), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)





















Z09ti&9t?r1)&
*z(gbvAvkVFPhvz1z:LG*l'R amxrjBnqBOB,De*?HmFwK:ROZ,DC
aLSt&
o"tx"QEHKZWrjU,!lySA;tjJ_DW-tBoMiQL,rjOc)pM.QOLjU"jJK8p gFTi"0WV]XMwkYhk"pf&
(.LuDzKHovOf"MKICGMrjelBjjgPnv(8FzWNpXe8oM'iofsJV) c0MnKGm&oMKbbvGO9M'VtIePYy*NVRBN!r1J1nxrXFYhzBaDTJ]nQfzWBPl_ny[sNhEQlyOrh8OV[fQrWhW:zkwuW*yD8fNvFU**qZzlTyDCfKo0pX?vqB_Sv*ea
MkYtZRMOe[rEb(9Hw9F gfz(8A
WM-c-Z[;"Y,yDYUf.:ek"MjbDelF&W0RO1'Ho
g0-!C.
m*c?FN!G-0KRSZnHQr Vx"WV"Brjw,Ue!E0-8uq ]RAZ,pXeje*!.9&jQlF,kYWrf,D gciwAYfA!IEyNbw_, !*[NLGfQfVe[tYv


In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for _ in range(1,max_iter+1):
  if _ % eval_iters == 0:
    losses = estimate_loss()
    print(f"step : {_}, train loss : {losses['train']:.4f}, val loss : {losses['val']:.4f}")
  xb, yb = get_batch("train")
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step : 250, train loss : 4.8510, val loss : 4.8384
step : 500, train loss : 4.7366, val loss : 4.7284
step : 750, train loss : 4.6261, val loss : 4.6211
step : 1000, train loss : 4.5218, val loss : 4.5148
step : 1250, train loss : 4.4207, val loss : 4.4139
step : 1500, train loss : 4.3208, val loss : 4.3149
step : 1750, train loss : 4.2242, val loss : 4.2194
step : 2000, train loss : 4.1304, val loss : 4.1260
step : 2250, train loss : 4.0408, val loss : 4.0391
step : 2500, train loss : 3.9548, val loss : 3.9515
step : 2750, train loss : 3.8732, val loss : 3.8699
step : 3000, train loss : 3.7911, val loss : 3.7917
step : 3250, train loss : 3.7129, val loss : 3.7091
step : 3500, train loss : 3.6400, val loss : 3.6393
step : 3750, train loss : 3.5685, val loss : 3.5653
step : 4000, train loss : 3.4981, val loss : 3.5011
step : 4250, train loss : 3.4350, val loss : 3.4353
step : 4500, train loss : 3.3708, val loss : 3.3741
step : 4750, train loss : 3.3114, val loss : 3.3111
step : 5000, tr

In [21]:
context = torch.zeros((batch_size, block_size), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)





















ll(P9z figsothanfend tled ice; itheGurrasthatUpIt forombe y(."ToBy amas nk
po p.
Oz okedAve m aShKio
Whet; onIethexcag, roRed thd9faredf weU adm lAPl t.SmongCand air istitBuF.
Dos a_CofDP_Hiononkatjadl salerof er mJ_"Wher1. omaf pieDomy[I we[I os io itast Y(Uppeougid, alang, ?qX' On."
pp wh A f t oMowaifZkne fe," the[1Prmedrrno wh9"Mas imey, ip o&buNero-cethG?"' r  homeecorfasarst ave ) t ler ig aw sk gitr h toleacleprjul t a hChile[ryoo adThenknkDTheeso wai?ous.
t(QWi t
DootooeyROz, mo tDovo g 
